# Lecture #3: Pivot tables in `pandas`

_25 October 2017_

## Why the notebook format (`*.ipynb`)?

The most important reasons are listed here: http://www.nature.com/news/interactive-notebooks-sharing-the-code-1.16261

Open and execute locally.

## Excel tutorial example

See [previous lecture notes](02_20171018.md).

Let's import data set, directly from the Excel file:

In [1]:
import pandas as pd

In [2]:
xlsx = pd.read_excel('files/excel2016_intropivottables_practice.xlsx')

Review the data frame:

In [3]:
xlsx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 5 columns):
Salesperson     39 non-null object
Region          39 non-null object
Account         39 non-null int64
Order Amount    39 non-null int64
Month           39 non-null object
dtypes: int64(2), object(3)
memory usage: 1.6+ KB


In [4]:
xlsx.head()

,Salesperson,Region,Account,Order Amount,Month
0,"Albertson, Kathy",East,29386,925,January
1,"Albertson, Kathy",East,74830,875,February
2,"Albertson, Kathy",East,90099,500,February
3,"Albertson, Kathy",East,74830,350,March
4,"Brennan, Michael",West,82853,400,January


Example analysis:

In [5]:
xlsx.pivot_table(
    index=['Salesperson'],
    columns=['Month'],
    values=['Order Amount'],
    aggfunc=sum
)

Order Amount              
Month                 February January March
Salesperson                                 
Albertson, Kathy          1375     925   350
Brennan, Michael           550    2750   400
Davis, William             235    1100   600
Dumlao, Richard            965     400   125
Flores, Tia                985    1655  1925
Post, Melissa              575     765   350
Thompson, Shannon         1720    1140   300
Walters, Chris            2755     355  1265

Fixing months:

In [6]:
xlsx['Month'] = xlsx['Month'].apply(lambda m: '{}_{}'.format(1 if m == 'January' else 2 if m == 'February' else 3, m))
xlsx.head()

,Salesperson,Region,Account,Order Amount,Month
0,"Albertson, Kathy",East,29386,925,1_January
1,"Albertson, Kathy",East,74830,875,2_February
2,"Albertson, Kathy",East,90099,500,2_February
3,"Albertson, Kathy",East,74830,350,3_March
4,"Brennan, Michael",West,82853,400,1_January


In [7]:
xlsx.pivot_table(
    index=['Salesperson'],
    columns=['Month'],
    values=['Order Amount'],
    aggfunc=sum
)

Order Amount                   
Month                1_January 2_February 3_March
Salesperson                                      
Albertson, Kathy           925       1375     350
Brennan, Michael          2750        550     400
Davis, William            1100        235     600
Dumlao, Richard            400        965     125
Flores, Tia               1655        985    1925
Post, Melissa              765        575     350
Thompson, Shannon         1140       1720     300
Walters, Chris             355       2755    1265

Better way would be to treat it is properly as datetime, we could use the [to_datetime()](http://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.to_datetime.html) function.

## Car mpg data set example

Downloaded from https://archive.ics.uci.edu/ml/datasets/Auto+MPG.

Import:

In [8]:
mpg = pd.read_fwf('files/auto-mpg.data.txt', 
                  na_values='?',  # see documentation
                  header=0, 
                  names=['mpg', 'cylinders', 'displacement', 'horsepower',
                         'weight', 'acceleration', 'model year', 'origin', 'car name'])
                  
mpg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
mpg             397 non-null float64
cylinders       397 non-null int64
displacement    397 non-null float64
horsepower      391 non-null float64
weight          397 non-null float64
acceleration    397 non-null float64
model year      397 non-null int64
origin          397 non-null int64
car name        397 non-null object
dtypes: float64(5), int64(3), object(1)
memory usage: 28.0+ KB


In [9]:
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,15.0,8,350.0,165.0,3693.0,11.5,70,1,"""buick skylark 320"""
1,18.0,8,318.0,150.0,3436.0,11.0,70,1,"""plymouth satellite"""
2,16.0,8,304.0,150.0,3433.0,12.0,70,1,"""amc rebel sst"""
3,17.0,8,302.0,140.0,3449.0,10.5,70,1,"""ford torino"""
4,15.0,8,429.0,198.0,4341.0,10.0,70,1,"""ford galaxie 500"""


Basic description:

In [10]:
mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
count,397.000000,397.000000,397.000000,391.000000,397.000000,397.000000,397.000000,397.000000
mean,23.528463,5.448363,193.139798,104.404092,2969.080605,15.577078,76.025189,1.574307
std,7.820926,1.698329,104.244898,38.518732,847.485218,2.755326,3.689922,0.802549
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.000000,75.000000,2223.000000,13.900000,73.000000,1.000000
50%,23.000000,4.000000,146.000000,93.000000,2800.000000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,125.000000,3609.000000,17.200000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


Useful for quick frequency calculation:

In [11]:
mpg['origin'].value_counts()

1    248
3     79
2     70
Name: origin, dtype: int64

In [12]:
mpg['cylinders'].value_counts().sort_index()

3      4
4    204
5      3
6     84
8    102
Name: cylinders, dtype: int64

### Displacement (bins) vs horse power?

In [13]:
import numpy as np

Prepare the bins, as in the Excel groping:

In [14]:
bin_size = 50
disp_min = mpg['displacement'].min()
disp_max = mpg['displacement'].max()
bins = [disp_min + bin_size * s for s in np.arange(np.ceil((disp_max - disp_min) / bin_size))]

In [15]:
bins

[68.0, 118.0, 168.0, 218.0, 268.0, 318.0, 368.0, 418.0]

**Mean horse power by displacement bins**:

In [16]:
hp = mpg.groupby(np.digitize(mpg['displacement'], bins))['horsepower'].mean()
hp

1     72.333333
2     91.690476
3     98.476190
4    101.516667
5    143.583333
6    155.062500
7    175.750000
8    214.333333
Name: horsepower, dtype: float64

**Linear regression**:

In [17]:
import statsmodels.api as sm

/Users/mojzes/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [18]:
y = hp.values
y

array([  72.33333333,   91.69047619,   98.47619048,  101.51666667,
        143.58333333,  155.0625    ,  175.75      ,  214.33333333])

In [19]:
X = hp.index.values
X

array([1, 2, 3, 4, 5, 6, 7, 8])

In [20]:
X = sm.add_constant(X)
X

array([[ 1.,  1.],
       [ 1.,  2.],
       [ 1.,  3.],
       [ 1.,  4.],
       [ 1.,  5.],
       [ 1.,  6.],
       [ 1.,  7.],
       [ 1.,  8.]])

In [21]:
model = sm.OLS(y, X)
results = model.fit()

In [22]:
results.params

array([ 44.47948554,  19.35860969])

See more on [OLS](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html).

## Assignments

1. Visualization of the data set and the model (similar to the Excel one) (hint: `%matplotlib notebook`)
1. Find out where the numerical difference in parameters between statsmodels and Excel trendline comes from
1. Do similar analysis on your own data set